In [1]:
from categories_download import get_categories
from requests import get
from bs4 import BeautifulSoup,NavigableString
import pandas as pd
import os
import re
import urllib.request, urllib.error, urllib.parse
import sys

In [187]:
class recipe_info:
    def __init__(self):
        self.recipe_name = None
        self.recipe_url = None
        self.recipe_img_link = None
        self.recipe_rating = None
        self.recipe_difficulty = None
        self.recipe_time = None
        self.preparation_time = None
        self.recipe_author = None
        self.recipe_steps = None
        self.recipe_ingredients = None
        self.recipe_portion = None
        self.recipe_accessories = None
        self.number_vote = None
        self.number_favorites = None
        self.variation_link = None
        self.variation_name = None
        self.machine_name = None
        self.machine_addtional_info = None
        self.addtional_categories_dic = None
        self.addtional_categories_name = None
        
    def item_info_download(self,item):
        #download author
        author_container = item.find_all('div',class_='item-author')[0]
        self.recipe_author = author_container.a.text
#         print(item_author)
        #download rating
        item_rating_container = item.find('div',class_ = 'item-rating clearfix')
        if (item_rating_container != None):
            self.recipe_rating = item_rating_container.div['data-average']
#             print(self.recipe_rating)
            #downloadd number of vote
            if (item_rating_container.span != None):
                vote_string = item_rating_container.span.text.strip()
                self.number_vote = re.findall('[0-9]+', vote_string)

        favourite_container = item.find('div',class_ = 'col-xs-2 favourite-inf')
        self.number_favorites = favourite_container.text.strip()
#         print(self.favorites)
        #download difficulty
        item_difficulty_container = item.find('div',class_ = 'col-xs-3 difficulty-inf')
        self.recipe_difficulty = item_difficulty_container.text.strip()
#         print(item_difficulty)
        #download cooking time
        item_time_container = item.find('div',class_ = 'col-xs-3 cooking-time-inf')
        self.recipe_time = item_time_container.text.strip()
#         print(item_time)
        #download name
        self.recipe_name = item.a['title']
#         print(item_name)
        #download image link
        self.recipe_img_link = item.img['src']
#         print(item_img_link)
        #download item url
        item_url_add = item.a['href']
        self.recipe_url = home_url+item_url_add
#         print(item_extend_url)
        

    def download_recipe_steps_ingredients(self, step_list,ingredient_list):
        self.recipe_steps = []
        if len(step_list) == 1:
            step_list_container = step_list[0].find_all('p')
            if (len(step_list_container) > 1):
                for step in step_list_container:
                    step_text = step.text.strip()
                    if step_text != '':
                        self.recipe_steps.append(step_text)
            elif(sum(1 for e in step_list[0].childGenerator())>4):
                for step in step_list[0].childGenerator():
                    if not (step and isinstance(step,NavigableString)):
                        continue
                    elif(str(step).strip() ==''):
                        continue
                    else:
                        self.recipe_steps.append(str(step).strip())
            else:
                self.recipe_steps.append(" ".join(step_list[0].text.split()))
        else:
#             try:
            for i in range(len(step_list)):
                self.recipe_steps.append(" ".join(step_list[i].text.split()))
#             except:
#                 for i in range(len(step_list)):
#                     self.recipe_steps.append(step_list[i].p.text)
        
        self.recipe_ingredients = []
        #append ingredients to dictionary
        for i in range(len(ingredient_list)):
             self.recipe_ingredients.append(" ".join(ingredient_list[i].text.split()))
        
    def recipe_download_details(self):
        recipe_response = get(self.recipe_url)
        recipe_html_soup = BeautifulSoup(recipe_response.text, 'html.parser')
        
        #download variations 
        variations_container = recipe_html_soup.find('div',class_= 'col-sm-12 sidebar-box variant-box')
        if (variations_container != None):
            self.variation_link = variations_container.a['href']
            self.variation_name = variations_container.a.text
        
        #find recipe steps list
        step_container = recipe_html_soup.find('ol', class_ = 'steps-list')
        step_list = step_container.find_all('li')

        #find ingredients list and portion
        ingredients_container = recipe_html_soup.find('div', class_ = 'ingredients')
        ingredient_list = ingredients_container.find_all('li')
        self.recipe_portion = ingredients_container.find('p',class_='padding-bottom-10').text.strip()

        #process recipe step and ingredient list
        self.download_recipe_steps_ingredients(step_list,ingredient_list)
        
        #accessories list download
        accessories_container = recipe_html_soup.find('div', class_ = 'accessories-list')
        if accessories_container == None:
            self.accessories = []
        else:
            tools_list = accessories_container.find_all("meta")
            accessories = []
            for tool in tools_list:
                accessories.append(tool.get("content"))
            self.recipe_accessories = [accessories]
        
        #download addtional info
        self.download_addtional_info(recipe_html_soup)
       
    def download_addtional_info(self,recipe_html_soup):
        addtional_info_container = recipe_html_soup.find('div',class_= 'additional-info')
        addtion_info_list = addtional_info_container.find_all('li')
        #preparation time dowload
        time_container = addtion_info_list[0]
        preparation_time_container = time_container.find_all('div',class_='smallText')
        self.preparation_time = preparation_time_container[0].find("span", id="preparation-time-final").text.strip()
        #cooking method
        cooking_method = preparation_time_container[1].text.strip()
        #machine and machine addtional info
        machine_contianer = addtion_info_list[2]
        self.machine_name = machine_contianer.h5.text
        self.machine_addtional_info = machine_contianer.find('div',class_="margin-top-10")
        if(self.machine_addtional_info!=None):
            self.machine_addtional_info = self.machine_addtional_info.text.strip()
        #addtional categroies
        addtional_categories_container = addtion_info_list[4]
        addtional_categories_list =addtional_categories_container.find_all('a',class_='catText preventDefault')
        self.addtional_categories_dic ={}
        self.addtional_categories_name = []
        for category in addtional_categories_list:
            self.addtional_categories_name.append(category.text)
            self.addtional_categories_dic[category.text] = category['href']
        
    def create_data_frame(self):
        #create data frame
        item_dic = {}
        item_dic['link'] = self.recipe_url
        item_dic['img link']= self.recipe_img_link
        item_dic['rating'] = self.recipe_rating
        item_dic['number vote'] = self.number_vote
        item_dic['difficulty'] = self.recipe_difficulty
        item_dic['cooking time'] = self.recipe_time
        item_dic['preparation time'] = self.preparation_time
        item_dic['author'] = self.recipe_author
        item_dic['steps'] = [self.recipe_steps]
        item_dic['ingredients'] = [self.recipe_ingredients]
        item_dic['accessories'] = self.recipe_accessories
        item_dic['number favorite'] = self.number_favorites
        item_dic['variation link'] = self.variation_link
        item_dic['variation name'] = self.variation_name
        item_dic['portion'] = self.recipe_portion
        item_dic['recipe created for'] = self.machine_name
        item_dic['machine addtional info'] = self.machine_addtional_info
#         item_dic['addtional categories'] = self.addtional_categories_dic
        item_dic['addtional categories'] = [self.addtional_categories_name]
        
        return item_dic
    
    
    def download_html(self,path):
        #save html to file
        response = urllib.request.urlopen(self.recipe_url)
        webContent = response.read()

        f = open(path+'/'+self.recipe_name+'.html', 'wb')
        f.write(webContent)
        f.close

In [6]:
home_url = "https://www.recipecommunity.com.au"
categories_info_file = get_categories()
categories_data = pd.read_csv(categories_info_file, index_col=0)

/******************************/
finish categories info download
/******************************/


In [189]:
dataset_columns = ['link','img link','rating','number vote','number favorite',
                   'difficulty','cooking time','portion','preparation time',
                   'author','accessories','steps','ingredients','recipe created for',
                   'machine addtional info','addtional categories','variation link','variation name']


categories_indices = categories_data.index
counter = 0
for category_index in categories_indices:
    print('/*******************************/')
    print('Start Download Category '+ str(category_index))
    print('/*******************************/')
    #creat folders and init file name
    os.mkdir(str(category_index))
    category_recipe_outfile = str(category_index) + '/'+str(category_index)+'.csv' 
    items_data = pd.DataFrame(columns=dataset_columns)
    #get item url
    category_url = categories_data['link'][category_index]
#     print(item_url)
    category_response = get(category_url)
    category_html_soup = BeautifulSoup(category_response.text, 'html.parser')
    
    pager_container = category_html_soup.find('div',class_= 'pager')
    lastpage_extend = pager_container.a['href']
    pager_number = re.findall('[0-9]+', lastpage_extend)[0]
    pager_number = int(pager_number)
    
    
    for current_page in range(1,pager_number+1): 
        
        page_url= category_url + '?page=' + str(current_page)
        print(page_url)
                        
        #download url contents and apply beautifulsoup
        item_response = get(page_url)
        item_html_soup = BeautifulSoup(item_response.text, 'html.parser')


        #find items on the first page
        items_container = item_html_soup.find_all('div', class_ = 'thumbnail result-recipe result-grid-display')
    #     print(len(items_container))
        
        for item in items_container:
            #init recipe class
            recipe_class = recipe_info()
            recipe_class.item_info_download(item)
            print("downloading "+ recipe_class.recipe_name)

            #download html
            html_path = str(category_index)
            recipe_class.download_html(html_path)
            
    #         #process recipe url
            recipe_class.recipe_download_details()

            #create recipe data frame
            item_dic = recipe_class.create_data_frame()
            item_info = pd.DataFrame(item_dic,index=[recipe_class.recipe_name])
            items_data = items_data.append(item_info)
            counter = counter +1
            if counter >= 1:
                break
    print('/*******************************/')
    print('Category '+ str(category_index)+' saved')
    print('/*******************************/')
    items_data.to_csv(category_recipe_outfile, index = True,sep=",")
        #finish current pager crawler
    #finish all pages
    break

/*******************************/
Start Download Category Baby food
/*******************************/
https://www.recipecommunity.com.au/categories/baby-food?page=1
downloading Baby Biscuits (sugar free)
https://www.recipecommunity.com.au/categories/baby-food?page=2
downloading Baby Veggie Frittata
https://www.recipecommunity.com.au/categories/baby-food?page=3
downloading Cous Cous Patties
https://www.recipecommunity.com.au/categories/baby-food?page=4
downloading Baby Custard - sugar free whole egg custard
https://www.recipecommunity.com.au/categories/baby-food?page=5
downloading baby food (4 months +)
https://www.recipecommunity.com.au/categories/baby-food?page=6
downloading Sweet Potato and Lamb Casserole
https://www.recipecommunity.com.au/categories/baby-food?page=7
downloading Sugar Free Strawberry Yogurt Muffins for Babies
https://www.recipecommunity.com.au/categories/baby-food?page=8
downloading Baby First Food
https://www.recipecommunity.com.au/categories/baby-food?page=9
downlo

In [182]:
items_data

,accessories,addtional categories,author,cooking time,difficulty,img link,ingredients,link,machine addtional info,number favorite,number vote,portion,preparation time,rating,recipe created for,steps,variation link,variation name
Baby Biscuits (sugar free),None,NaN,Jessscott,13min,easy,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,"[60 gram rolled oats, 40 gram pitted dates, 25...",https://www.recipecommunity.com.au/baby-food-r...,Please note that the TM5 mixing bowl has a lar...,14,41,28 portion(s),5min,4.71,TM 5,"[Preheat oven to 180 degrees, Line 2 baking tr...",None,None
Baby Veggie Frittata,"[Spatula, Measuring cup]",NaN,Passionfruit Avenue,35min,easy,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,"[350 g mixed vegetables, roughly chopped. I us...",https://www.recipecommunity.com.au/baby-food-r...,Please note that the TM5 mixing bowl has a lar...,7,14,8 portion(s),10min,4.79,TM 5,[Preheat oven to 180c. Line a small (15cm squa...,None,None
Cous Cous Patties,"[Spatula, Measuring cup]",NaN,nataliematt@aapt.net.au,25min,easy,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,"[50 grams cheese, 150 grams Vegetables of choi...",https://www.recipecommunity.com.au/baby-food-r...,None,0,5,12 portion(s),5min,4.6,TM 31,[Cut 50g of cheese into cubes and grate in TM3...,None,None
Baby Custard - sugar free whole egg custard,None,NaN,JulieHill,10min,easy,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,"[500 g Milk, I used half half formula &amp; fc...",https://www.recipecommunity.com.au/baby-food-r...,Please note that the TM5 mixing bowl has a lar...,2,2,0 portion(s),10min,5,TM 5,"[Put all ingredients into bowl, cook for 6 min...",None,None
baby food (4 months +),[Measuring cup],NaN,mashakasha,4min,easy,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,"[2 Apples, small, 1 carrot, small, 2 water, MC]",https://www.recipecommunity.com.au/baby-food-r...,None,3,1,4 portion(s),1min,5,TM 31,[wash and cut apples into quarters. de-seed th...,None,None
Sweet Potato and Lamb Casserole,None,NaN,liz mancini,5min,--,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,"[500 g Lamb, Boneless, trimmed of fat and dice...",https://www.recipecommunity.com.au/baby-food-r...,None,12,16,4 portion(s),5min,4.31,TM 31,[Place onion herbs and garlic in TM bowl and c...,None,None
Sugar Free Strawberry Yogurt Muffins for Babies,[Spatula],NaN,DJTwiggy,17min,easy,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,"[230 grams self raising wholemeal flour, 120 g...",https://www.recipecommunity.com.au/baby-food-r...,Please note that the TM5 mixing bowl has a lar...,2,8,24 piece(s),2min,3.5,TM 5,"[Preheat oven to 180 degrees., Add flour, coco...",/baking-sweet-recipes/sugar-free-strawberry-bl...,Sugar Free Strawberry & Blueberry Coconut Muff...
Baby First Food,"[Varoma, Simmering basket, Spatula, Measuring ...",NaN,Peppylou91,25min,easy,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,"[200 grams Meat e.g steak, chicken breast, Tri...",https://www.recipecommunity.com.au/baby-food-r...,Please note that the TM5 mixing bowl has a lar...,0,5,24 piece(s),5min,4.2,TM 5,[1. Prepare all vegetables and fruit. Add wate...,None,None
Gluten Free Baby Cereal with Quinoa,None,NaN,Thermomix with Nadia,10min,easy,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,"[50 g Quinoa flakes, 100 g water, 50 g water, ...",https://www.recipecommunity.com.au/baby-food-r...,None,2,3,4 portion(s),2min,4.33,TM 31,[Place Quinoa Flakes and 100g water into your ...,/starters-recipes/gluten-free-bocconcini-and-r...,Gluten Free Bocconcini and Risotto Balls
baby food (6 months +),[Measuring cup],NaN,mashakasha,7min,easy,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,"[135 grams apple, 75 grams Honey Dew melon, 1 ...",https://www.recipecommunity.com.au/baby-food-r...,None,1,1,4 portion(s),7min,4,TM 31,"[1. wash, cut and de-seed your apples and melo...",None,None


In [183]:
item_dic

{'link': 'https://www.recipecommunity.com.au/baby-food-recipes/tuna-and-sweet-potato-patties/9gv9gjlm-b4c93-315962-cfcd2-0qg9srf2',
 'img link': 'https://d2mkh7ukbp9xav.cloudfront.net/assets/20200101-111111/bundles/tmrcfront/images/nopicture_rectangle.jpg',
 'rating': '0',
 'number vote': None,
 'difficulty': 'easy',
 'cooking time': '50min',
 'preparation time': '10min',
 'author': 'mrsplappy',
 'steps': [['1. Preheat oven to 180degrees and oil a lined tray',
   '2. Peel and roughly chop up sweet potato and place in basket in TM bowl (filled half way with water). Steam for 20 minutes, varoma temperature, speed 1.',
   '3. Remove basket from TM and drain water out of bowl. Tip sweet potato back in TM bowl, along with all other ingredients.',
   '4. Mix for 20 seconds, speed 6. Repeat until smooth.',
   '5. Roll into bite size balls and place on tray. Flatten with a spoon.',
   '6. Bake until golden brown.']],
 'ingredients': [['1 sweet potato, large',
   '1 can tuna, in springwater (95

In [184]:
item_info

,link,img link,rating,number vote,difficulty,cooking time,preparation time,author,steps,ingredients,accessories,number favorite,variation link,variation name,portion,recipe created for,machine addtional info,addtional categories
Tuna and Sweet Potato Patties,https://www.recipecommunity.com.au/baby-food-r...,https://d2mkh7ukbp9xav.cloudfront.net/assets/2...,0,None,easy,50min,10min,mrsplappy,[1. Preheat oven to 180degrees and oil a lined...,"[1 sweet potato, large, 1 can tuna, in springw...",[Simmering basket],0,None,None,10 piece(s),TM 5,Please note that the TM5 mixing bowl has a lar...,NaN


In [10]:
item_url = 'https://www.recipecommunity.com.au/categories/baby-food?page=6'

In [11]:
recipe_response = get('https://www.recipecommunity.com.au/baby-food-recipes/beef-and-three-vegetable-puree/1etj02hr-9d870-628097-beea0-13numfvs')
recipe_html_soup = BeautifulSoup(recipe_response.text, 'html.parser')

#find recipe steps list
step_container = recipe_html_soup.find('ol', class_ = 'steps-list')
step_list = step_container.find_all('li')

#find ingredients list
ingredients_container = recipe_html_soup.find('div', class_ = 'ingredients')
ingredient_list = ingredients_container.find_all('li')

recipe_steps = []
try:
    for i in range(len(step_list)):
        recipe_steps.append(" ".join(step_list[i].text.split()))
except:
    for i in range(len(step_list)):
        recipe_steps.append(step_list[i].p.text)

recipe_ingredients = []
#append ingredients to dictionary
for i in range(len(ingredient_list)):
     recipe_ingredients.append(" ".join(ingredient_list[i].text.split()))



In [29]:
recipe_steps = []
if len(step_list) == 1:
    step_list_container = step_list[0].find_all('p')
    if (len(step_list_container) > 1):
        for step in step_list_container:
            step_text = step.text.strip()
            if step_text != '':
                recipe_steps.append(step_text)
    elif(sum(1 for e in step_list[0].childGenerator())>4):
        for step in step_list[0].childGenerator():
            if not (step and isinstance(step,NavigableString)):
                continue
            elif(str(step).strip() ==''):
                continue
            else:
                recipe_steps.append(str(step).strip())
    else:
        for i in range(len(step_list)):
            recipe_steps.append(" ".join(step_list[i].text.split()))

else:
#             try:
    for i in range(len(step_list)):
        recipe_steps.append(" ".join(step_list[i].text.split()))

In [133]:
recipe_response = get('https://www.recipecommunity.com.au/baby-food-recipes/apple-kiwi-and-clove-puree-for-reusable-pouches/kl3uxs3v-0745c-386041-cfcd2-c0f9cw9z')
# recipe_response = get('https://www.recipecommunity.com.au/baby-food-recipes/baby-biscuits-sugar-free/vdttw1rs-09448-352970-cfcd2-icxafnee')
recipe_html_soup = BeautifulSoup(recipe_response.text, 'html.parser')

#download variations 
variations_container = recipe_html_soup.find('div',class_= 'col-sm-12 sidebar-box variant-box')
if (variations_container != None):
    variation_link = variations_container.a['href']
    variation_name = variations_container.a.text

#find recipe steps list
step_container = recipe_html_soup.find('ol', class_ = 'steps-list')
step_list = step_container.find_all('li')

#find ingredients list
ingredients_container = recipe_html_soup.find('div', class_ = 'ingredients')
ingredient_list = ingredients_container.find_all('li')

#process recipe step and ingredient list
# download_recipe_steps_ingredients(step_list,ingredient_list)

#accessories list download
accessories_container = recipe_html_soup.find('div', class_ = 'accessories-list')
if accessories_container == None:
    accessories = []
else:
    tools_list = accessories_container.find_all("meta")
    accessories = []
    for tool in tools_list:
        accessories.append(tool.get("content"))
    recipe_accessories = [accessories]
    
addtional_info_container = recipe_html_soup.find('div',class_= 'additional-info')
addtion_info_list = addtional_info_container.find_all('li')

In [157]:
time_container = addtion_info_list[0]
time = time_container.h5.text
preparation_time_container = time_container.find_all('div',class_='smallText')
preparation_time = preparation_time_container[0].find("span", id="preparation-time-final").text.strip()

''

''

In [137]:
cooking_method_container = preparation_time_container[1].text.strip()
print(cooking_method_container)

Baking/Cooking


In [140]:
machine_contianer = addtion_info_list[2]
print(machine_contianer)

<li>
<div class="step-container">
<div class="edit-icon"></div>
<span class="step-number">8</span>
<div class="appliances-list">
<ul>
<li class="media appliance-version no-border">
<div class="media-left">
<img alt="Appliance TM 31 image" class="media-object" data-holder-rendered="true" data-src="holder.js/50x50" src="//tmrcv3-common.s3.amazonaws.com/appliances/tm31_small.png" style=""/>
</div>
<div class="media-body">
<span class="smallText">
                Recipe is created for
              </span>
<h5 class="media-heading">TM 31</h5>
</div>
</li>
</ul>
</div>
</div>
</li>


In [142]:
machine_name = machine_contianer.h5.text
machine_addtional_info = machine_contianer.find('div',class_="margin-top-10")
if(machine_addtional_info!=None):
    machine_addtional_info = machine_addtional_info.text
print(machine_addtional_info)

None


In [159]:
addtional_categories_container = addtion_info_list[4]
addtional_categories_list =addtional_categories_container.find_all('a',class_='catText preventDefault')
addtional_categories_dic ={}
for category in addtional_categories_list:
    addtional_categories_dic[category.text] = category['href']
addtional_categories_dic

{}

In [160]:
tips_container = recipe_html_soup.find('div',class_='tips')
tips = tips_container.find('p').text
print(tips_container)


<div class="tips">
<div class="step-container">
<div class="edit-icon"></div>
<span class="step-number">11</span>
<h2 class="h3">Tip</h2>
<p><p> </p> </p>
</div>
</div>


In [161]:
tips = tips_container.find('p').text
print(tips)

In [155]:
portion= ingredients_container.find('p',class_='padding-bottom-10').text
print(portion)


 6 portion(s)

